In [30]:
import os
import torch
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from deep_fields import data_path
from torch.distributions import Normal, MultivariateNormal
from torch.distributions import Bernoulli, Beta, Poisson
from scipy.stats import invwishart, bernoulli

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

%matplotlib inline

In [46]:
from deep_fields import data_path

data_dir = os.path.join(data_path,"raw\merton_poisson_covariance")
data_dir

'c:\\users\\cesar\\desktop\\projects\\general\\deep_random_fields\\data\\raw\\merton_poisson_covariance'

In [40]:
data_path

'c:\\users\\cesar\\desktop\\projects\\general\\deep_random_fields\\data'

In [16]:
number_of_processes = 4
number_of_realizations = 100
jump_arrival_alpha = 1.
jump_arrival_beta = 1.

jump_size_a = 1.
jump_size_b = 1.

# JUMPS
arrivals_intensity_prior = Beta(jump_arrival_alpha, jump_arrival_beta)
arrivals_indicator_prior = Bernoulli(arrivals_intensity_prior.sample())

nu = number_of_processes + 1.
Psi = np.random.rand(number_of_processes, number_of_processes)
Psi = np.dot(Psi, Psi.transpose())
covariance_jump_prior = invwishart(nu,Psi)

In [17]:
# sample from intensity prior
arrivals_intensity = arrivals_intensity_prior.sample(sample_shape=(number_of_processes,))
arrivals_indicator_prior = Bernoulli(arrivals_intensity)
arrivals_indicator = arrivals_indicator_prior.sample(sample_shape=(number_of_realizations,))

In [19]:
# sample from jump size prior
jump_covariance = covariance_jump_prior.rvs()
jump_covariance =  torch.Tensor(jump_covariance)
jump_mean_prior = torch.ones(number_of_processes) * jump_size_a
jump_mean_prior = MultivariateNormal(jump_mean_prior, jump_size_b *jump_covariance)
jump_mean =jump_mean_prior.sample()

In [21]:
jump_distribution = MultivariateNormal(jump_mean, jump_covariance)
jumps_size = jump_distribution.sample(sample_shape=(number_of_realizations,))

In [26]:
(jumps_size*arrivals_indicator)[:10,:]

tensor([[0.0000, 1.7876, 1.4282, 1.6375],
        [0.0000, 0.0000, 0.0000, 0.0000],
        [1.2495, 0.0000, 1.0690, 1.7523],
        [0.0000, 0.0000, 1.3190, 0.4499],
        [0.0000, 0.0000, 1.9418, 0.0000],
        [0.0000, 0.0000, 0.8718, 1.7254],
        [0.5733, 0.0000, 1.4003, 0.0680],
        [0.5968, 0.0000, 0.0000, 0.1783],
        [0.0000, 0.0000, 0.0000, 0.3185],
        [1.3335, 0.0000, 0.0000, 0.0000]])

In [28]:
arrivals_indicator[:10,:]

tensor([[0., 1., 1., 1.],
        [0., 0., 0., 0.],
        [1., 0., 1., 1.],
        [0., 0., 1., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 0., 1.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.]])

In [ ]:
# sample from jump size prior
jump_covariance = covariance_jump_prior.rvs()
jump_covariance =  torch.Tensor(jump_covariance)
jump_mean_prior = torch.ones(self.number_of_processes) * self.jump_size_a
self.jump_mean_prior = MultivariateNormal(jump_mean_prior, self.jump_size_b *jump_covariance)
jump_mean = self.jump_mean_prior.sample()

self.jump_distribution = MultivariateNormal(jump_mean, jump_covariance)
jumps_size = self.jump_distribution.sample(sample_shape=(self.number_of_realizations,))

# covariance -------------------
diffusion_covariance = self.diffusion_covariance_normalization*self.covariance_diffusion_prior.rvs()

diffusion_covariance = torch.Tensor(diffusion_covariance)
diffusion_mean_prior = torch.ones(self.number_of_processes) * self.returns_mean_a
self.diffusion_mean_prior = MultivariateNormal(diffusion_mean_prior, self.returns_mean_b * diffusion_covariance)
expected_return = self.diffusion_mean_prior.sample()

# are we missing time scale Delta t? Assumed == 1
diffusive_log_returns = expected_return[None, :] + \
                        MultivariateNormal(torch.zeros(diffusion_covariance.shape[0]), diffusion_covariance).sample(
                            sample_shape=(self.number_of_realizations,))

log_returns = diffusive_log_returns + jumps_size * arrivals_indicator[:, None]

data_loader = {"arrivals_intensity": arrivals_intensity.item(),
               "arrivals_indicator": arrivals_indicator,
               "jump_mean": jump_mean,
               "jump_covariance": jump_covariance,
               "jump_size": jumps_size,
               "diffusive_log_returns": diffusive_log_returns,
               "log_returns": log_returns,
               "diffusion_covariance":diffusion_covariance,
               "expected_returns": expected_return}